In [ ]:
from math import sqrt
from typing import Tuple

import numpy as np
from matplotlib import pyplot as plt
from numpy.typing import NDArray
from scipy import stats

In [ ]:
N = 6
S = 8
I1 = 5
I2 = 4

alpha_1 = 0.1
alpha_2 = 0.01

tau = 1 / (1 + I1)
p1 = (alpha_1 ** (1 - tau)) * (alpha_2 ** tau)
p2 = 5 * p1 / np.sqrt(S)

N1 = 10 * (2 + N)
N2 = 100 * (2 + N)
l = int((I2 + 2.5) / 2)

thetas = {
    1: (0, 1),
    2: (S, N),
    3: (N, S + 2),
    4: (S, S + N),
    5: (-N, S + 4),
}


def compute_pareto_mean(scale: float, shape: float) -> float:
    return stats.pareto(b=shape, scale=scale).mean().item()


def compute_pareto_variance(scale: float, shape: float) -> float:
    return stats.pareto(b=shape, scale=scale).var().item()


def compute_theta(_mu_0: float, _v0: float) -> Tuple[float, float]:
    a1 = (_v0 + sqrt(_v0 ** 2 + _v0 * _mu_0 ** 2)) / _v0
    a2 = (_v0 - sqrt(_v0 ** 2 + _v0 * _mu_0 ** 2)) / _v0
    a = max(a1, a2)
    assert a > 2
    xm = _mu_0 * (a - 1) / a
    assert xm > 0
    return xm, a


def compute_g(_theta: Tuple[float, float], num_samples: int) -> NDArray:
    return stats.pareto.rvs(b=_theta[1], scale=_theta[0], size=num_samples)


def plot_samples(samples: NDArray) -> None:
    plt.figure(figsize=(8, 6))
    plt.hist(samples, bins=50, density=True, alpha=0.75, color="blue", edgecolor="black")
    plt.title("Distribution")
    plt.xlabel("Value")
    plt.ylabel("Density")
    plt.show()


print(f"""
N = {N}
S = {S}
I1 = {I1}
I2 = {I2}
alpha_1 = {alpha_1}
alpha_2 = {alpha_2}
tau = {tau}
p1 = {p1}
p2 = {p2}
N1 = {N1}
N2 = {N2}
l = {l}
thetas = {thetas}
""")

In [ ]:
theta_0 = thetas[l]
mu_0 = compute_pareto_mean(*theta_0)
v0 = compute_pareto_variance(*theta_0)
theta_1 = compute_theta(mu_0, N * v0)
theta_2 = compute_theta(mu_0 + 2 * sqrt(v0), v0 / S)

print(f"""
theta_0 = {theta_0}
mu_0 = {mu_0:.4f}
v0 = {v0:.4f}
theta_1 = {theta_1}
theta_2 = {theta_2}
""")

In [ ]:
G0 = compute_g(theta_0, num_samples=N2)
G1 = compute_g(theta_1, num_samples=N2)
G2 = compute_g(theta_2, num_samples=N2)

In [ ]:
FY1 = (1 - p1) * G0 + p1 * G1
plot_samples(FY1)

In [ ]:
FY2 = (1 - p2) * G0 + p2 * G2
plot_samples(FY2)